# CRF+LSTM

keras 2.2.4

tensorflow 1.13

pip install git+https://www.github.com/keras-team/keras-contrib.git

In [3]:
import re

In [7]:
char_vocab_path = "CRF/data/char_vocabs.txt" # 字典文件
train_data_path = 'data/train_data/train_data_000' # 训练数据
#train_data_path = './data/train_data' # 训练数据
test_data_path = 'data/train_data/train_data_000' # 测试数据

special_words = ['<PAD>', '<UNK>'] # 特殊词表示

# "BIO"标记的标签
#label2idx = {"O": 0,
#             "B-PER": 1, "I-PER": 2,
#             "B-LOC": 3, "I-LOC": 4,
#             "B-ORG": 5, "I-ORG": 6
#            }
label2idx = {'O': 0,
             'DISEASE': 1, 'DISEASE_GROUP': 2,
             'DRUG_DOSAGE': 3, 'DRUG_EFFICACY': 4,
             'DRUG_INGREDIENT': 5, 'DRUG_TASTE': 6,
             'FOOD_GROUP':7, 'PERSON_GROUP':8,
             'SYMPTOM':9, 'SYNDROME':10
            }

# 索引和BIO标签对应
idx2label = {idx: label for label, idx in label2idx.items()}

# 读取字符词典文件
with open(char_vocab_path, "r", encoding="utf8") as fo:
    char_vocabs = [line.strip() for line in fo]
char_vocabs = special_words + char_vocabs

# 字符和索引编号对应
idx2vocab = {idx: char for idx, char in enumerate(char_vocabs)}
vocab2idx = {char: idx for idx, char in idx2vocab.items()}

In [8]:
# 读取训练语料
def read_corpus(corpus_path, vocab2idx, label2idx):
    with open(corpus_path, encoding='utf-8') as fr:
        lines = fr.readlines()

    sent_, tag_ = [], []
    for letter in lines:
        [char,label,_] = re.split('\t|\n',letter)
        sent_.append(char)
        tag_.append(label)

    sent_ids = [vocab2idx[char] if char in vocab2idx else vocab2idx['<UNK>'] for char in sent_]
    tag_ids = [label2idx[label] if label in label2idx else 0 for label in tag_]
    return sent_ids, tag_ids

# 加载训练集
train_datas, train_labels = read_corpus(train_data_path, vocab2idx, label2idx)
# 加载测试集
test_datas, test_labels = read_corpus(test_data_path, vocab2idx, label2idx)


In [10]:
train_datas = []
train_labels = []
for i in range(1000):
    train_data_path_i = 'data/train_data/train_data_%03d'%i
    train_datas_i, train_labels_i = read_corpus(train_data_path_i, vocab2idx, label2idx)
    train_datas.append(train_datas_i)
    train_labels.append(train_labels_i)
    #if i%10==0:
    #    print(i)

In [11]:
test_datas = train_datas
test_labels = train_labels

In [12]:
print(train_datas[50])
print([idx2vocab[idx] for idx in train_datas[50]])
print(train_labels[50])
print([idx2label[idx] for idx in train_labels[50]])

[0, 589, 2644, 451, 5801, 5844, 5253, 966, 2953, 5253, 406, 3769, 182, 2659, 998, 4719, 451, 5801, 3647, 4101, 5980, 4717, 2178, 4793, 3900, 4463, 4702, 2008, 2494, 218, 6267, 3903, 2022, 1357, 6802, 2377, 6573, 1374, 5253, 1635, 6756, 3903, 5253, 4435, 5169, 3900, 911, 6009, 6802, 4539, 4020, 1635, 6756, 651, 5253, 2545, 6214, 966, 5253, 3160, 1357, 6040, 2545, 6214, 182, 0, 589, 2644, 6315, 3383, 4392, 2770, 406, 3769, 182, 2659, 998, 868, 1338, 719, 6315, 6756, 1534, 1588, 6007, 6009, 6802, 1338, 6573, 1367, 6756, 1534, 1588, 2288, 2494, 966, 6315, 287, 5988, 911, 6009, 6802, 723, 3556, 651, 3648, 723, 2065, 2654, 3903, 2994, 3630, 1338, 1363, 182, 0, 589, 2644, 463, 4669, 406, 3769, 182, 2659, 998, 868, 2276, 2223, 32, 12, 3020, 1297, 785, 280, 4377, 2178, 4793, 3903, 2000, 2002, 4669, 2327, 376, 1367, 6756, 5528, 228, 5738, 3020, 5983, 966, 5528, 4934, 5738, 3020, 5983, 504, 3903, 782, 6573, 6817, 1338, 719, 1192, 3022, 765, 4084, 2178, 4793, 2118, 2002, 4669, 2327, 1591, 1367, 67

In [ ]:
import numpy as np
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Masking, Embedding, Bidirectional, LSTM, Dense, Input, TimeDistributed, Activation
from keras.preprocessing import sequence
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras_contrib.metrics import crf_viterbi_accuracy
from keras import backend as K
K.clear_session()

EPOCHS = 30
BATCH_SIZE = 64
EMBED_DIM = 32
HIDDEN_SIZE = 16
MAX_LEN = 500
VOCAB_SIZE = len(vocab2idx)
CLASS_NUMS = len(label2idx)
print(VOCAB_SIZE, CLASS_NUMS)

print('padding sequences')
train_datas = sequence.pad_sequences(train_datas, maxlen=MAX_LEN)
train_labels = sequence.pad_sequences(train_labels, maxlen=MAX_LEN)
test_datas = sequence.pad_sequences(test_datas, maxlen=MAX_LEN)
test_labels = sequence.pad_sequences(test_labels, maxlen=MAX_LEN)
print('x_train shape:', train_datas.shape)
print('x_test shape:', test_datas.shape)

train_labels = keras.utils.to_categorical(train_labels, CLASS_NUMS)
test_labels = keras.utils.to_categorical(test_labels, CLASS_NUMS)
print('trainlabels shape:', train_labels.shape)
print('testlabels shape:', test_labels.shape)

## BiLSTM+CRF模型构建
inputs = Input(shape=(MAX_LEN,), dtype='int32')
x = Masking(mask_value=0)(inputs)
x = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(x)
x = Bidirectional(LSTM(HIDDEN_SIZE, return_sequences=True))(x)
x = TimeDistributed(Dense(CLASS_NUMS))(x)
outputs = CRF(CLASS_NUMS)(x)
model = Model(inputs=inputs, outputs=outputs)
model.summary()

model.compile(loss=crf_loss, optimizer='adam', metrics=[crf_viterbi_accuracy])
model.fit(train_datas, train_labels, epochs=EPOCHS, verbose=1, validation_split=0.1)

score = model.evaluate(test_datas, test_labels, batch_size=BATCH_SIZE)
print(model.metrics_names)
print(score)

# save model
model.save("./model/ch_ner_model.h5")

Using TensorFlow backend.





6874 11
padding sequences
x_train shape: (1000, 500)
x_test shape: (1000, 500)
trainlabels shape: (1000, 500, 11)
testlabels shape: (1000, 500, 11)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
masking_1 (Masking)          (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 32)           219968    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500, 32)           6272      
_________________________________________________________________
time_distributed_1 (TimeDist (None, 500, 11)           363       
___________________________

In [ ]:
from keras.models import load_model
import numpy as np

maxlen = 500
sentence = " 北京同仁堂科技发展股份有限公司制药厂  1.忌食辛辣，少进油腻。 2.感冒发热病人不宜服用。 3.有高血压、心脏病、肝病、糖尿病、肾病等慢性病严重者应在医师指导下服用。 4.伴有月经紊乱者，应在医师指导下服用。 5.眩晕症状较重者，应及时去医院就诊。 6.服药2周症状无缓解，应去医院就诊。 7.对本品过敏者禁用，过敏体质者慎用。 8.本品性状发生改变时禁止使用。 9.请将本品放在儿童不能接触的地方。 10.如正在使用其他药品，使用本品前请咨询医师或药师。  本品为浅黄色至棕黄色颗粒，气微香，味微苦。  滋养肝肾、宁心安神。用于更年期综合症属阴虚肝旺症，症见烘热汗出，头晕耳鸣，失眠多梦，五心烦热，腰背酸痛，大便干燥，心烦易怒，舌红少苔，脉弦细或弦细 开水冲服。一次1袋(12g)，一日3次。  如与其他药物同时使用可能会发生药物相互作用，详情请咨询医师或药师。  12g*10袋/盒  用于更年期综合症属阴虚肝旺症  铝塑复合膜包装，每袋装12克，每盒装10袋。  非处方药物（甲类）,中药保护品种二级  12g*10袋/盒  用于更年期综合症属阴虚肝旺更年期综合症气微香，味微苦。"

sent_chars = list(sentence)
sent2id = [vocab2idx[word] if word in vocab2idx else vocab2idx['<UNK>'] for word in sent_chars]

sent2id_new = np.array([[0] * (maxlen-len(sent2id)) + sent2id[:maxlen]])
y_pred = model.predict(sent2id_new)
y_label = np.argmax(y_pred, axis=2)
y_label = y_label.reshape(1, -1)[0]
y_ner = [idx2label[i] for i in y_label][-len(sent_chars):]

print(idx2label)
print(sent_chars)
print(sent2id)
print(y_ner)

In [ ]:
list(zip(sent_chars,y_ner))